## Create a conda environment

In [ ]:
!wget https://repo.anaconda.com/archive/Anaconda3-5.2.0-Linux-x86_64.sh && bash Anaconda3-5.2.0-Linux-x86_64.sh -bfp /usr/local

## Install dependencies

In [ ]:
!yes | conda install pytorch==1.0.0 torchvision==0.2.1 cuda100 -c pytorch
!pip install opencv-python==3.4.5.20
!pip install -q youtube-dl
!pip install ffmpeg
!pip install gdown

## Clone the github repo

In [5]:
!git clone https://github.com/vaib-saxena/Slow-Fast-pytorch-implementation.git

Cloning into 'Slow-Fast-pytorch-implementation'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 253 (delta 2), reused 5 (delta 2), pack-reused 245
Receiving objects: 100% (253/253), 65.03 MiB | 11.65 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [6]:
%cd /content/Slow-Fast-pytorch-implementation

/content/Slow-Fast-pytorch-implementation


## Download pretrained weights

In [ ]:
import os
from os.path import exists, join, basename
project_name = "/content/Slow-Fast-pytorch-implementation"

yolo_pretrained_weight_dir = project_name
if not exists(join(yolo_pretrained_weight_dir, 'yolov3.weights')):
  file_id = '1zAh58ZEbtOmt8tvqEbh2p7jCdt1Q7xbK'
  !cd {yolo_pretrained_weight_dir} && gdown "https://drive.google.com/uc?id={file_id}"
    
deepsort_pretrained_weight_dir = join(project_name, 'deep/checkpoint')
if not exists(join(deepsort_pretrained_weight_dir, 'ckpt.t7')):
  file_id = '1_qwTWdzT9dWNudpusgKavj_4elGgbkUN'
  !cd {deepsort_pretrained_weight_dir} && curl -Lb ./cookie "https://drive.google.com/uc?export=download&id={file_id}" -o ckpt.t7

slowfast_pretrained_weight_dir = project_name
if not exists(join(slowfast_pretrained_weight_dir, 'slowfast_weight.pth')):
  file_id = '1wteH0LxrGV9_Z1PO6YC-2m9Qte9fMGao'
  !cd {slowfast_pretrained_weight_dir} && gdown "https://drive.google.com/uc?id={file_id}"

## Build the extension

In [ ]:
!python /content/Slow-Fast-pytorch-implementation/support/setup.py build develop

In [3]:
%cd /content/Slow-Fast-pytorch-implementation

/content/Slow-Fast-pytorch-implementation


## Mount your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

## Downloading the video from drive

In [ ]:
## you can provide your own Google drive ID(last digits of shareable link) 
drive_id='1CgjWcFfqt_m4lUdNshd70EVOEAtfle0j'

!gdown https://drive.google.com/uc?id=drive_id

# -ss requires the start of the video and -t requires the duration of the video, new.mp4 is the name of the video that can be changed accordingly
!ffmpeg -ss 00:00:00.00 -i /content/Slow-Fast-pytorch-implementation/final_low_bitrate.mp4 -t 00:00:10.00 -c copy new.mp4

## Run the code on the video file

In [ ]:
#Change reso(resolution) values(512,1024,2048 etc.) as per the need, higher the resolution, higher the accuracy 
#but it compromises with speed. Furthermore, the value of confidence can be changed(0 to 1) 
#it refers to the algorithm's confidence for predictions
#--video refers to the location of the video you want to run

!python video_demo.py --reso=416 --confidence=0.5 --video="/content/Slow-Fast-pytorch-implementation/new.mp4"

## Creating the output as video file(saving in Google Drive)

In [1]:
import cv2
import numpy as np
import glob

#saving the individual frames in the following folder
path = '/content/drive/My Drive/Activity_Outputs/frames/'

img_array = []
for count in range(1000):
  filename = '/content/drive/My Drive/Activity_Outputs/frames/' + str(count) + '.jpg'
  img = cv2.imread(filename)
  height, width, layers = img.shape
  size = (width,height)
  img_array.append(img)

#set your own frame rate
frame = 30
out = cv2.VideoWriter('/content/drive/My Drive/Activity_Outputs/output.mp4',cv2.VideoWriter_fourcc(*'DIVX'), frame, size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

## Download video and csv

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download video file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id_video = 'REPLACE_WITH_YOUR_FILE_ID'
downloaded = drive.CreateFile({'id': file_id_video})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

# Download csv file based on its file ID.
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id_csv = 'REPLACE_WITH_YOUR_FILE_ID'
downloaded = drive.CreateFile({'id': file_id_csv})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))